In [1]:
import pandas as pd
import numpy as np
import random as rd

from rdkit import Chem
from rdkit.Chem import AllChem, rdFingerprintGenerator

In [2]:
# # take a subset.
# cell2ind = pd.read_csv('../DrugCell/data/cell2ind.txt', header=None, sep='\t')
# # take a whole things
# gene2ind = pd.read_csv('../DrugCell/data/gene2ind.txt', header=None, sep='\t')
# # take a subset.
# drug2ind = pd.read_csv('../DrugCell/data/drug2ind.txt', header=None, sep='\t')
# # take a subset.
# cell2mutation = pd.read_csv('../DrugCell/data/cell2mutation.txt', header=None, sep=',')
# # take a subset.
# drug2fingerprint = pd.read_csv('../DrugCell/data/drug2fingerprint.txt', header=None, sep=',')
# # take a whole things
# drugcell_ont = pd.read_csv('../DrugCell/data/drugcell_ont.txt', header=None, sep='\t')
# # make train, test and validation
# rcellminer = pd.read_csv('../DrugCell/data/rcellminer_test.txt', header=None, sep='\t')

In [3]:
# indexes = list(rcellminer.index)
# rd.Random(42).shuffle(indexes)

In [4]:
# test = indexes[round(len(indexes)*0.8):]
# tmp = indexes[:round(len(indexes)*0.8)]
# val = tmp[:len(test)]
# train = tmp[len(test):]

In [5]:
# rcellminer.iloc[test].to_csv(
#     '../data/test.txt', sep='\t', 
#     header=None, index=None
# )

In [6]:
# rcellminer.iloc[val].to_csv(
#     '../data/val.txt', sep='\t', 
#     header=None, index=None
# )

In [7]:
# rcellminer.iloc[train].to_csv(
#     '../data/train.txt', sep='\t', 
#     header=None, index=None
# )

In [8]:
ls ./DrugCell/data/

cell2ind.txt*          pubchem_id.csv
cell2mutation.txt      rcellminer_test_remove_negative_values.txt
drug2fingerprint.txt   rcellminer_test_set_threshold.txt
drug2ind_rcell.txt     rcellminer_test.txt
drug2ind.txt*          sanger_cell_lines.txt
drugcell_ont.txt       sanger_to_ccle.csv
drugcell_test.txt      sanger_to_ccle.xlsx
drugcell_train.txt     smiles_by_nsc.csv
drugcell_val.txt       SMILES_from_PubchemID.txt
gene2ind.txt*          test.csv
mfp.txt                test_rcell.txt
nci60Act.csv           test.txt
nci60GeneMut.csv       train_rcell.txt
out.txt                train.txt
out_.txt               val_rcell.txt
pubchem_id_by_nsc.csv  val.txt


In [9]:
# Create PubChem_id list
pubchem_id = pd.read_csv('./DrugCell/data/pubchem_id_by_nsc.csv', index_col=0).dropna()
pubchem_id['PUBCHEM_ID'].astype(int).to_csv('./DrugCell/data/pubchem_id.csv', index=None, header=None)
pubchem_id.shape

(16522, 1)

In [10]:
#  Get SMILES from PubChem ID  https://pubchem.ncbi.nlm.nih.gov/idexchange/idexchange.cgi
SMILES = pd.read_table('./DrugCell/data/SMILES_from_PubchemID.txt', header=None)
SMILES.index = pubchem_id.index
SMILES = SMILES.drop(0, axis=1)
SMILES.columns = ['SMILES']
SMILES.shape

(16522, 1)

## Drug to Index

In [11]:
SMILES.reset_index(drop=True).to_csv(
    './DrugCell/data/drug2ind_rcell.txt', 
    header=None,
    sep='\t'
)

## mogan finger printing

In [12]:
mfp = np.array([
 np.array(
  AllChem.GetMorganFingerprintAsBitVect(
    Chem.MolFromSmiles(i), 
    useChirality=True, 
    radius=2, 
    nBits=2048
  )) for i in SMILES['SMILES']
])

[10:44:49] WARNING: not removing hydrogen atom without neighbors
[10:44:49] WARNING: not removing hydrogen atom without neighbors


In [13]:
mfp = pd.DataFrame(mfp)
mfp.head()

,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
mfp.to_csv(
    './DrugCell/data/mfp.txt',
    index=None,
    header=None
)

In [15]:
nci60Act = pd.read_csv('./DrugCell/data/nci60Act.csv', index_col=0)

# need to change the name to match to the table.
nci60Act = nci60Act.rename(
    columns={
        'BR:HS 578T': 'BR:HS 578 T', 
        'BR:T-47D': 'BR:T47D',
        'CNS:SF-295': 'CNS:SF295',
        'CNS:SF-268': 'CNS:SF268',
        'CNS:SF-539': 'CNS:SF539',
        'CNS:SNB-19': 'CNS:SNB19',
        'CO:HT29': 'CO:HT 29',
        'CO:SW-620': 'SW620',
        'LE:HL-60(TB)': 'LE:HL-60',
        'ME:LOX IMVI': 'ME:LOXIMVI',
        'LC:A549/ATCC': 'LC:A549',
        'OV:IGROV1': 'OV:IGROV 1',
        'RE:RXF 393': 'RE:RXF-393',
        'RE:TK-10': 'RE:TK10' 
    }
)

# These are not listed in the table and train data
nci60Act = nci60Act.drop([
    'ME:MDA-N', 
    'OV:NCI/ADR-RES',
    'RE:UO-31'
], axis=1)

In [16]:
nci60Act

,BR:MCF7,BR:MDA-MB-231,BR:HS 578 T,BR:BT-549,BR:T47D,CNS:SF268,CNS:SF295,CNS:SF539,CNS:SNB19,CNS:SNB-75,...,OV:SK-OV-3,PR:PC-3,PR:DU-145,RE:786-0,RE:A498,RE:ACHN,RE:CAKI-1,RE:RXF-393,RE:SN12C,RE:TK10
1,-0.271314,-0.303539,-0.815183,-0.231499,1.934731,-0.357577,-0.719253,-0.380720,-1.281589,-0.175915,...,-0.868240,-0.403983,-0.434885,-0.519867,-1.648059,1.657273,-0.269717,0.002290,-0.390689,-0.379420
17,-0.354110,-0.304675,-0.222024,1.483613,1.509397,0.335572,0.424922,1.140166,-0.941890,0.330808,...,-0.492953,1.302516,-0.941890,0.521200,-0.329639,-0.941890,0.717851,-0.239018,-0.315533,-0.761559
89,NaN,NaN,NaN,NaN,NaN,-0.184194,-1.429903,-0.165433,-1.429903,-0.216723,...,-0.126557,NaN,NaN,0.368433,-0.655707,-0.144006,0.743888,-0.435620,-0.184805,-0.101929
185,NaN,NaN,NaN,NaN,NaN,0.539343,NaN,0.230402,-0.765829,-1.125208,...,-2.151708,NaN,NaN,-0.061612,NaN,NaN,1.675353,0.918722,1.391604,-0.904724
295,-0.264586,-0.264586,-0.264586,-0.264586,-0.264586,-0.264586,-0.264586,-0.264586,-0.264586,-0.264586,...,-0.264586,-0.264586,-0.264586,4.822657,-0.264586,-0.264586,-0.264586,-0.264586,-0.264586,-0.264586
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
900911,-0.167151,-0.167151,-0.167151,-0.167151,-0.167151,-0.167151,-0.167151,-0.167151,-0.167151,-0.167151,...,-0.167151,-0.167151,-0.167151,-0.167151,-0.167151,-0.167151,-0.167151,1.594059,-0.167151,-0.167151
900922,-0.169786,-0.169786,-0.169786,-0.169786,-0.169786,-0.169786,-0.169786,-0.169786,-0.169786,-0.169786,...,-0.169786,-0.169786,-0.169786,-0.169786,-0.169786,-0.169786,-0.169786,-0.169786,-0.169786,-0.169786
900964,-0.158754,-0.158754,-0.158754,-0.158754,-0.158754,-0.158754,-0.158754,-0.158754,-0.158754,-0.158754,...,-0.158754,-0.158754,NaN,-0.158754,-0.158754,-0.158754,-0.158754,-0.158754,-0.158754,-0.158754
900974,-0.132453,-0.132453,-0.132453,NaN,-0.132453,-0.132453,-0.132453,-0.132453,-0.132453,-0.132453,...,-0.132453,-0.132453,-0.132453,-0.132453,-0.132453,-0.132453,-0.132453,NaN,-0.132453,-0.132453


In [17]:
col = []
for i in nci60Act.columns:
    tmp = i.split(':')
    if len(tmp) == 2:
        col.append(tmp[1])
    else:
        col.append(tmp[0])

nci60Act.columns = col

In [18]:
sanger_cell_lines = pd.read_csv('./DrugCell/data/sanger_to_ccle.csv', index_col=0)
cell2ind = list(pd.read_table('./DrugCell/data/cell2ind.txt', header=None)[1])

In [19]:
ccle_dict = {
    i.upper():j for i,j in zip(
        sanger_cell_lines['Sanger name'], 
        sanger_cell_lines['CCLE Label']
    )
}

In [20]:
nci60Act.columns = [
    ccle_dict[i.replace(' ', '-')] for i in nci60Act.columns
]
nci60Act = nci60Act[list(set(cell2ind) & set(nci60Act.columns))]

In [21]:
nci60Act.head()

,T47D_BREAST,SF539_CENTRAL_NERVOUS_SYSTEM,COLO205_LARGE_INTESTINE,MALME3M_SKIN,SKMEL28_SKIN,A549_LUNG,IGROV1_OVARY,NCIH23_LUNG,A498_KIDNEY,UACC257_SKIN,...,OVCAR8_OVARY,RPMI8226_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,NCIH460_LUNG,K562_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,SN12C_KIDNEY,SF295_CENTRAL_NERVOUS_SYSTEM,HOP92_LUNG,MOLT4_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,786O_KIDNEY,DU145_PROSTATE
1,1.934731,-0.380720,0.008989,-0.233261,-0.443281,-0.826223,0.149074,-0.600872,-1.648059,-0.407149,...,-0.413599,1.504554,-0.700613,1.183415,-0.390689,-0.719253,2.110059,1.540716,-0.519867,-0.434885
17,1.509397,1.140166,-0.041714,-0.756130,-0.777004,-0.941890,1.353076,-0.925414,-0.329639,-0.061915,...,-0.572734,1.698491,-0.941890,-0.037619,-0.315533,0.424922,-0.078754,1.686372,0.521200,-0.941890
89,NaN,-0.165433,0.152000,0.318373,0.176544,-0.433001,-0.181827,-0.312738,-0.655707,-0.070365,...,0.185498,1.191198,-0.183471,0.035400,-0.184805,-1.429903,-0.068839,0.549935,0.368433,NaN
185,NaN,0.230402,0.174485,-0.917027,0.011812,0.500858,-0.852569,0.170420,NaN,-0.807890,...,0.095648,-0.024337,NaN,0.477536,1.391604,NaN,-0.038789,1.675353,-0.061612,NaN
295,-0.264586,-0.264586,-0.264586,-0.264586,-0.264586,-0.264586,-0.264586,-0.264586,-0.264586,-0.264586,...,3.362653,NaN,-0.264586,-0.264586,-0.264586,-0.264586,-0.264586,-0.264586,4.822657,-0.264586


In [22]:
base = pd.DataFrame()
for i in nci60Act.columns:
    tmp = pd.DataFrame((nci60Act[i].dropna()))
    tmp.columns = ['Drug Response']
    tmp['Cell Line'] = [i]*len(tmp)
    tmp = pd.merge(tmp, SMILES, left_index=True, right_index=True)
    base = pd.concat([base, tmp])
    
base = base[['Cell Line', 'SMILES', 'Drug Response']]
base = base.reset_index(drop=True)

In [23]:
print(len(base['Cell Line'].unique()))
print(len(base['SMILES'].unique()))

55
16522


In [24]:
pd.DataFrame(base['Cell Line'].value_counts())

,Cell Line
A549_LUNG,16168
OVCAR8_OVARY,16145
U251MG_CENTRAL_NERVOUS_SYSTEM,16087
HCT15_LARGE_INTESTINE,16082
HCT116_LARGE_INTESTINE,16079
SW620_LARGE_INTESTINE,16073
SF295_CENTRAL_NERVOUS_SYSTEM,16024
KM12_LARGE_INTESTINE,16006
COLO205_LARGE_INTESTINE,16003
SF268_CENTRAL_NERVOUS_SYSTEM,15993


In [25]:
indexes = list(base.index)
rd.Random(42).shuffle(indexes)

In [26]:
test = indexes[round(len(indexes)*0.8):]
tmp = indexes[:round(len(indexes)*0.8)]
val = tmp[:len(test)]
train = tmp[len(test):]

In [27]:
train = base.iloc[train]
test = base.iloc[test]
val = base.iloc[val]

In [28]:
print('train: ' + str(train.shape))
print('val: ' + str(val.shape))
print('test: ' + str(test.shape))

train: (498877, 3)
val: (166292, 3)
test: (166292, 3)


In [29]:
df = pd.merge(
    pd.merge(
        pd.DataFrame(train['Cell Line'].value_counts()), 
        pd.DataFrame(val['Cell Line'].value_counts()), 
        left_index=True, right_index=True
    ), pd.DataFrame(test['Cell Line'].value_counts()), 
    left_index=True, right_index=True
)
df.columns = ['train', 'val', 'test']
df.loc['total'] = np.sum(df, axis=0) 

In [30]:
df

,train,val,test
HCT116_LARGE_INTESTINE,9795,3170,3114
SF268_CENTRAL_NERVOUS_SYSTEM,9670,3199,3124
A549_LUNG,9643,3266,3259
COLO205_LARGE_INTESTINE,9642,3222,3139
U251MG_CENTRAL_NERVOUS_SYSTEM,9622,3287,3178
HT29_LARGE_INTESTINE,9618,3122,3132
UACC257_SKIN,9609,3040,3229
KM12_LARGE_INTESTINE,9605,3196,3205
HCT15_LARGE_INTESTINE,9602,3215,3265
OVCAR5_OVARY,9599,3220,3099


In [31]:
test.to_csv(
    './DrugCell/data/test_rcell.txt', sep='\t', 
    header=None, index=None
)

val.to_csv(
    './DrugCell/data/val_rcell.txt', sep='\t', 
    header=None, index=None
)

train.to_csv(
    './DrugCell/data/train_rcell.txt', sep='\t', 
    header=None, index=None
)